In [1]:
import numpy as np
import tensornetwork as tn
import tensorflow as tf

# Layers

In [16]:
class QuantumDMRGLayer(tf.keras.layers.Layer):
    def __init__(self, dimvec, pos_label, nblabels, bond_len, unihigh):
        super(QuantumDMRGLayer, self).__init__()
        self.dimvec = dimvec
        self.pos_label = pos_label
        self.nblabels = nblabels
        self.m = bond_len
        self.unihigh = unihigh
        self.construct_tensornetwork()

    def construct_tensornetwork(self):
        self.mps_tf_vars = [None] * self.dimvec
        for i in range(self.dimvec):
            if i == 0 or i == self.dimvec - 1:
                self.mps_tf_vars[i] = tf.Variable(tf.random.uniform(shape=(2, self.m),
                                                                    minval=0,
                                                                    maxval=self.unihigh),
                                                  name='mps_node_{}'.format(i),
                                                  trainable=True)
            elif i == self.pos_label:
                self.mps_tf_vars[i] = tf.Variable(tf.random.uniform(shape=(2, self.m, self.m, self.nblabels),
                                                                    minval=0,
                                                                    maxval=self.unihigh),
                                                  name='mps_node_{}'.format(i),
                                                  trainable=True)
            else:
                self.mps_tf_vars[i] = tf.Variable(tf.random.uniform(shape=(2, self.m, self.m),
                                                                    minval=0,
                                                                    maxval=self.unihigh),
                                                  name='mps_node_{}'.format(i),
                                                  trainable=True)

        # model nodes
        self.nodes = [
            tn.Node(self.mps_tf_vars[i], name='node{}'.format(i), backend='tensorflow')
            for i in range(self.dimvec)
        ]

        # input nodes
        cosx = np.random.uniform(size=self.dimvec)
        self.input_nodes = [tn.Node(np.array([cosx[i], np.sqrt(1-cosx[i]*cosx[i])]), name='input{}'.format(i), backend='tensorflow') 
                            for i in range(self.dimvec)]

    def infer_single_datum(self, input):
        for i in range(self.dimvec):
            self.input_nodes[i].tensor = input[i, :]
        edges = [self.nodes[0][1] ^ self.nodes[1][1]]
        for i in range(1, self.dimvec - 1):
            edges.append(self.nodes[i][2] ^ self.nodes[i + 1][1])

        input_edges = [self.nodes[i][0] ^ self.input_nodes[i][0] for i in range(self.dimvec)]

        final_node = tn.contractors.auto(self.nodes + self.input_nodes,
                                         output_edge_order=[self.nodes[self.pos_label][3]])
        return final_node.tensor

    def call(self, inputs):
        return tf.vectorized_map(self.infer_single_datum, inputs)

In [20]:
dmrg_layer = QuantumDMRGLayer(dimvec=784, pos_label=392, nblabels=10, bond_len=20, unihigh=0.075)

In [21]:
cosx = np.random.uniform(size=784)

inputs = np.array([np.array([cosx, np.sqrt(1-cosx*cosx)]).T])

In [23]:
%time dmrg_layer(inputs)

CPU times: user 55 s, sys: 2.49 s, total: 57.5 s
Wall time: 1min 25s


<tf.Tensor: id=288917, shape=(1, 10), dtype=float32, numpy=
array([[9.8851476e-14, 9.6363483e-14, 9.5171511e-14, 9.2989418e-14,
        9.5006007e-14, 9.3354476e-14, 1.0011910e-13, 9.5419590e-14,
        9.3016130e-14, 9.4698832e-14]], dtype=float32)>

In [29]:
quantum_dmrg_model = tf.keras.Sequential(
    tf.keras.Input(shape=(None, 784, 2)),
    QuantumDMRGLayer(dimvec=784, pos_label=392, nblabels=10, bond_len=20, unihigh=0.05)
)

OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

In [27]:
isinstance(dmrg_layer, tf.keras.layers.Layer)

True